In [1]:
# -*- coding: utf-8 -*-
# 导入相应的库
from browsermobproxy import Server
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
from pprint import pprint
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
# browsermob-proxy.bat就是刚才下载的文件解压得来，路径根据自己实际的进行修改，我这里是解压放到了Dp盘；port为端口，根据自己情况进行修改，我这里顺便写入了8739
server = Server('../../scrapy/browsermob-proxy-2.1.4/bin/browsermob-proxy')
server.start()
proxy = server.create_proxy()

In [2]:
chrome_options = Options()
# 无浏览器界面，根据自己实际情况是否使用，我这里使用浏览器界面，故注释了这行代码
# chrome_options.add_argument('--headless')

# # 不加载图片，提升运行速度
# chrome_options.add_argument('blink-settings=imagesEnabled=false') 
# chrome_options.add_argument(r"user-data-dir=//Users//zhouyang//Library//Application Support//Google//Chrome//User Data")    # 
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36')
# chrome_options.add_argument("--start-maximized")  # 开始最大化
# chrome_options.add_argument("--test-type")
# chrome_options.add_argument("--ignore-certificate-errors")  # 忽略证书错误
 
# chrome_options.add_argument("--disable-popup-blocking")  # 禁用弹出拦截
# chrome_options.add_argument("no-sandbox")  # 取消沙盒模式
# chrome_options.add_argument("no-default-browser-check")  # 禁止默认浏览器检查
# chrome_options.add_argument("about:histograms")
# chrome_options.add_argument("about:cache")
 
# chrome_options.add_argument("disable-extensions")  # 禁用扩展
# chrome_options.add_argument("disable-glsl-translator")  # 禁用GLSL翻译
 
# chrome_options.add_argument("disable-translate")  # 禁用翻译
# chrome_options.add_argument("--disable-gpu")  # 谷歌文档提到需要加上这个属性来规避bug
# chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("--hide-scrollbars")  # 隐藏滚动条, 应对一些特殊页面
# chrome_options.add_argument("blink-settings=imagesEnabled=false")  # 不加载图片, 提升速度
# 需要加上这句代码，否则有些网页会被浏览器拦截了。实际如何，看大家在用的过程的情况
chrome_options.add_argument('--ignore-certificate-errors')

# 设置代理
chrome_options.add_argument('--proxy-server={0}'.format(proxy.proxy))

driver = webdriver.Chrome(options=chrome_options)


In [4]:
driver.get('https://passport.mafengwo.cn/')

mm_login = driver.find_element(by=By.XPATH, value='//p[@data-type="password"]')
mm_login.click()
time.sleep(1)

user_input = driver.find_element(by=By.XPATH, value='//input[@id="login_ppt"]')
pw_input = driver.find_element(by=By.XPATH, value='//input[@id="pwd"]')
button_login = driver.find_element(by=By.XPATH, value='//button[@id = "_js_loginBtn"]')
time.sleep(1)
# user_input.clear()
user_input.send_keys('18005817635')
# pw_input.click()
pw_input.send_keys('zhouyang201202')
# time.sleep(1)
button_login.click()
time.sleep(1)

In [7]:
proxy.new_har('html_list', options={'captureContent': True})
for i in range(1,22):
    # 也可以以目标URL配置一个新的HAR并获取HAR内容
    base_url = f'https://www.mafengwo.cn/hotel/14336/#avl=0&distance=10000&price=-&feature=0&fav=0&sort=comment-DESC&page={i}'
    driver.get(base_url)
    driver.refresh()
    # 暂停几秒等待页面加载完成，防止拿不到结果
proxy.wait_for_traffic_to_stop(1, 30)
result = proxy.har
# pprint(result)
# 假设 result 是一个包含请求信息的字典，具体结构类似于您提供的 'result['log']['entries']'。

url_prefix = "https://www.mafengwo.cn/hotel/ajax.php?iMddId=14336&iAreaId=-1&iRegionId=-1&"

with open('./file/request_url.txt', 'a') as u:
    for ent in result['log']['entries']:
        url = ent['request']['url']
        if url.startswith(url_prefix):
            u.write(url + '\n')

In [120]:

server.stop()
driver.quit()


In [173]:
proxy.new_har('html_list', options={'captureContent': True})
url = 'https://www.mafengwo.cn/jd/14336/gonglve.html'
driver.get(url)
proxy.wait_for_traffic_to_stop(1, 30)
next_element = driver.find_element(by=By.XPATH, value='//a[@class="pi pg-next" and @title="后一页"]')
next_element.click()

<selenium.webdriver.remote.webelement.WebElement (session="1c2a2657912d68b84f6997bef319dc44", element="0BA016D6F6B2D4CF6F1EE97C6CD32E32_element_463")>

In [9]:
proxy.new_har('html_list', options={'captureContent': True})
url = 'https://www.mafengwo.cn/jd/14336/gonglve.html'
driver.get(url)
proxy.wait_for_traffic_to_stop(1, 30)
while True:
    try:
        # 尝试找到符合条件的元素
        time.sleep(3)
        next_element = driver.find_element(by=By.XPATH, value='//a[@class="pi pg-next" and @title="后一页"]')
        next_element.click()
        
    except NoSuchElementException:
        # 当找不到元素时，捕获NoSuchElementException异常，并跳出while循环
        break
time.sleep(5)
result = proxy.har

In [14]:
proxy.new_har('html_list', options={'captureContent': True})
url = 'https://www.mafengwo.cn/jd/14336/gonglve.html'
driver.get(url)
proxy.wait_for_traffic_to_stop(1, 30)
result = proxy.har


In [10]:
# 创建一个空字典，用于存储提取的数据
data_dict = {}
for ent in result['log']['entries']:
    if ent['request']['url'] == 'https://www.mafengwo.cn/ajax/router.php':
        ent_html = json.loads(ent['response']['content']['text'])['data']['list']
        # 使用BeautifulSoup解析HTML文本
        soup = BeautifulSoup(ent_html, 'html.parser')
        # 查找所有的<li>元素
        li_elements = soup.find_all('li')

        # 遍历每个<li>元素，提取href和title属性，并存入字典
        for li in li_elements:
            a_tag = li.find('a')
            href = 'https://www.mafengwo.cn' + a_tag['href']
            title = a_tag['title']
            data_dict[title] = href
data_dict

{'茂名中国第一滩': 'https://www.mafengwo.cn/poi/6328481.html',
 '浪漫海岸国际旅游度假区': 'https://www.mafengwo.cn/poi/7055059.html',
 '放鸡岛海上度假公园': 'https://www.mafengwo.cn/poi/5426463.html',
 '高州仙人洞风景区': 'https://www.mafengwo.cn/poi/5426498.html',
 '茂名森林公园': 'https://www.mafengwo.cn/poi/6328482.html',
 '露天矿生态公园': 'https://www.mafengwo.cn/poi/25940321.html',
 '石根山风景旅游区': 'https://www.mafengwo.cn/poi/5428537.html',
 '茂名市博物馆': 'https://www.mafengwo.cn/poi/6328486.html',
 '南山寺观音殿': 'https://www.mafengwo.cn/poi/5350757.html',
 '海滨公园': 'https://www.mafengwo.cn/poi/6328491.html',
 '玉湖国家水利风景区': 'https://www.mafengwo.cn/poi/7055063.html',
 '根子镇': 'https://www.mafengwo.cn/poi/6726267.html',
 '博贺港': 'https://www.mafengwo.cn/poi/6328483.html',
 '御水古温泉': 'https://www.mafengwo.cn/poi/5427389.html',
 '高州市宝光塔': 'https://www.mafengwo.cn/poi/5398038.html',
 '高州水库': 'https://www.mafengwo.cn/poi/8073203.html',
 '小东江摩天轮': 'https://www.mafengwo.cn/poi/70042936.html',
 '高州冼太庙': 'https://www.mafengwo.cn/poi/33634200.html',
 '

In [11]:
with open('file/data.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_dict, json_file, ensure_ascii=False, indent=4)

In [ ]:
with open('./file/request_url.txt', 'a') as u:
    for ent in result['log']['entries']:
        url = ent['response']

In [4]:
import requests
from pyquery import PyQuery as pq
# 数据初始化
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
}
url = 'https://www.mafengwo.cn/hotel/14336/#avl=0&distance=10000&price=-&feature=0&fav=0&sort=comment-DESC&page=1'
res = requests.get(url,headers=header).text
doc =pq(res)